# Recognize named entities on Twitter with LSTMs

In this assignment, you will use a recurrent neural network to solve Named Entity Recognition (NER) problem. NER is a common task in natural language processing systems. It serves for extraction such entities from the text as persons, organizations, locations, etc. In this task you will experiment to recognize named entities from Twitter.

For example, we want to extract persons' and organizations' names from the text. Than for the input text:

    Ian Goodfellow works for Google Brain

a NER model needs to provide the following sequence of tags:

    B-PER I-PER    O     O   B-ORG  I-ORG

Where *B-* and *I-* prefixes stand for the beginning and inside of the entity, while *O* stands for out of tag or no tag. Markup with the prefix scheme is called *BIO markup*. This markup is introduced for distinguishing of consequent entities with similar types.

A solution of the task will be based on neural networks, particularly, on Bi-Directional Long Short-Term Memory Networks (Bi-LSTMs).

### Libraries

For this task you will need the following libraries:
 - [Tensorflow](https://www.tensorflow.org) — an open-source software library for Machine Intelligence.
 
In this assignment, we use Tensorflow 1.15.0. You can install it with pip:

    !pip install tensorflow==1.15.0
     
 - [Numpy](http://www.numpy.org) — a package for scientific computing.
 
If you have never worked with Tensorflow, you would probably need to read some tutorials during your work on this assignment, e.g. [this one](https://www.tensorflow.org/tutorials/recurrent) could be a good starting point. 

### Data

The following cell will download all data required for this assignment into the folder `week2/data`.

In [1]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    ! wget https://raw.githubusercontent.com/hse-aml/natural-language-processing/master/setup_google_colab.py -O setup_google_colab.py
    import setup_google_colab
    setup_google_colab.setup_week2()

import sys
sys.path.append("..")
from common.download_utils import download_week2_resources

download_week2_resources()

### Load the Twitter Named Entity Recognition corpus

We will work with a corpus, which contains tweets with NE tags. Every line of a file contains a pair of a token (word/punctuation symbol) and a tag, separated by a whitespace. Different tweets are separated by an empty line.

The function *read_data* reads a corpus from the *file_path* and returns two lists: one with tokens and one with the corresponding tags. You need to complete this function by adding a code, which will replace a user's nickname to `<USR>` token and any URL to `<URL>` token. You could think that a URL and a nickname are just strings which start with *http://* or *https://* in case of URLs and a *@* symbol for nicknames.

In [2]:
def read_data(file_path):
    tokens = []
    tags = []
    
    tweet_tokens = []
    tweet_tags = []
    for line in open(file_path, encoding='utf-8'):
        line = line.strip()
        if not line:
            if tweet_tokens:
                tokens.append(tweet_tokens)
                tags.append(tweet_tags)
            tweet_tokens = []
            tweet_tags = []
        else:
            token, tag = line.split()
            if token.startswith("https://") or token.startswith("http://"):
                token = '<URL>'
            if token.startswith("@"):
                token = '<USR>'
            tweet_tokens.append(token)
            tweet_tags.append(tag)
            
    return tokens, tags

And now we can load three separate parts of the dataset:
 - *train* data for training the model;
 - *validation* data for evaluation and hyperparameters tuning;
 - *test* data for final evaluation of the model.

In [3]:
train_tokens, train_tags = read_data('data/train.txt')
validation_tokens, validation_tags = read_data('data/validation.txt')
test_tokens, test_tags = read_data('data/test.txt')

You should always understand what kind of data you deal with. For this purpose, you can print the data running the following cell:

In [4]:
for i in range(3):
    for token, tag in zip(train_tokens[i], train_tags[i]):
        print('%s\t%s' % (token, tag))
    print()

RT	O
<USR>	O
:	O
Online	O
ticket	O
sales	O
for	O
Ghostland	B-musicartist
Observatory	I-musicartist
extended	O
until	O
6	O
PM	O
EST	O
due	O
to	O
high	O
demand	O
.	O
Get	O
them	O
before	O
they	O
sell	O
out	O
...	O

Apple	B-product
MacBook	I-product
Pro	I-product
A1278	I-product
13.3	I-product
"	I-product
Laptop	I-product
-	I-product
MD101LL/A	I-product
(	O
June	O
,	O
2012	O
)	O
-	O
Full	O
read	O
by	O
eBay	B-company
<URL>	O
<URL>	O

Happy	O
Birthday	O
<USR>	O
!	O
May	O
Allah	B-person
s.w.t	O
bless	O
you	O
with	O
goodness	O
and	O
happiness	O
.	O



### Prepare dictionaries

To train a neural network, we will use two mappings: 
- {token}$\to${token id}: address the row in embeddings matrix for the current token;
- {tag}$\to${tag id}: one-hot ground truth probability distribution vectors for computing the loss at the output of the network.

Now you need to implement the function *build_dict* which will return {token or tag}$\to${index} and vice versa. 

In [5]:
from collections import defaultdict

In [25]:
def build_dict(tokens_or_tags, special_tokens):
    """
        tokens_or_tags: a list of lists of tokens or tags
        special_tokens: some special tokens
    """
    # Create a dictionary with default value 0
    tok2idx = defaultdict(lambda: 0)
    idx2tok = []
    
    for item in special_tokens:
        if item not in idx2tok:
            idx2tok.append(item)
    for item in tokens_or_tags:
        for token_or_tag in item:
            if token_or_tag not in idx2tok:
                idx2tok.append(token_or_tag)
    for i, added_item in enumerate(idx2tok):
        tok2idx[added_item] = i
    
    return tok2idx, idx2tok

After implementing the function *build_dict* you can make dictionaries for tokens and tags. Special tokens in our case will be:
 - `<UNK>` token for out of vocabulary tokens;
 - `<PAD>` token for padding sentence to the same length when we create batches of sentences.

In [26]:
special_tokens = ['<PAD>', '<UNK>']
special_tags = ['O']

# Create dictionaries 
token2idx, idx2token = build_dict(train_tokens + validation_tokens, special_tokens)
tag2idx, idx2tag = build_dict(train_tags, special_tags)

The next additional functions will help you to create the mapping between tokens and ids for a sentence. 

In [9]:
def words2idxs(tokens_list):
    return [token2idx[word] for word in tokens_list]

def tags2idxs(tags_list):
    return [tag2idx[tag] for tag in tags_list]

def idxs2words(idxs):
    return [idx2token[idx] for idx in idxs]

def idxs2tags(idxs):
    return [idx2tag[idx] for idx in idxs]

### Generate batches

Neural Networks are usually trained with batches. It means that weight updates of the network are based on several sequences at every single time. The tricky part is that all sequences within a batch need to have the same length. So we will pad them with a special `<PAD>` token. It is also a good practice to provide RNN with sequence lengths, so it can skip computations for padding parts. We provide the batching function *batches_generator* readily available for you to save time. 

In [16]:
import numpy as np
def batches_generator(batch_size, tokens, tags,
                      shuffle=True, allow_smaller_last_batch=True):
    """Generates padded batches of tokens and tags."""
    
    n_samples = len(tokens)
    if shuffle:
        order = np.random.permutation(n_samples)
    else:
        order = np.arange(n_samples)

    n_batches = n_samples // batch_size
    if allow_smaller_last_batch and n_samples % batch_size:
        n_batches += 1

    for k in range(n_batches):
        batch_start = k * batch_size
        batch_end = min((k + 1) * batch_size, n_samples)
        current_batch_size = batch_end - batch_start
        x_list = []
        y_list = []
        max_len_token = 0
        for idx in order[batch_start: batch_end]:
            x_list.append(words2idxs(tokens[idx]))
            y_list.append(tags2idxs(tags[idx]))
            max_len_token = max(max_len_token, len(tags[idx]))
            
        # Fill in the data into numpy nd-arrays filled with padding indices.
        x = np.ones([current_batch_size, max_len_token], dtype=np.int32) * token2idx['<PAD>']
        y = np.ones([current_batch_size, max_len_token], dtype=np.int32) * tag2idx['O']
        for n in range(current_batch_size):
            utt_len = len(x_list[n])
            x[n, :utt_len] = x_list[n]
            y[n, :utt_len] = y_list[n]
        yield (x, y)

## Build a recurrent neural network

This is the most important part of the assignment. Here we will specify the network architecture based on TensorFlow building blocks. It's fun and easy as a lego constructor! We will create an LSTM network which will produce probability distribution over tags for each token in a sentence. To take into account both right and left contexts of the token, we will use Bi-Directional LSTM (Bi-LSTM). Dense layer will be used on top to perform tag classification.  
### Modified Section
I have used tensorflow 2.0 to build the network.
This does not require sessions, and the masking of the error function corresponding to PAD is done automatically. The model has the same architecture as the one provided, 

In [17]:
import warnings
warnings.filterwarnings("ignore")
import logging
logging.getLogger('tensorflow').disabled = True
import tensorflow as tf
tf.autograph.set_verbosity(0)
tf.get_logger().disabled = True
from tensorflow.keras.layers import LSTM, Bidirectional, Dense, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
class BiLSTMModel(Model):
    def __init__(self, vocabulary_size, embedding_dim, n_hidden_rnn, n_tags, drop_prob):
        super(BiLSTMModel, self).__init__()
        self.embeddor = Embedding(vocabulary_size, embedding_dim, mask_zero=True)
        self.BiRNN = Bidirectional(LSTM(n_hidden_rnn, recurrent_dropout=drop_prob, return_sequences=True))
        self.head = Dense(n_tags, activation = None)
    def call(self, inputs):
        embedded = self.embeddor(inputs)
        rnn_outs = self.BiRNN(embedded)
        logits = self.head(rnn_outs)
        return logits

We finished with necessary methods of our BiLSTMModel model and almost ready to start experimenting.

### Evaluation 
To simplify the evaluation process we provide two functions for you:
 - *predict_tags*: uses a model to get predictions and transforms indices to tokens and tags;
 - *eval_conll*: calculates precision, recall and F1 for the results.

In [18]:
from evaluation import precision_recall_f1

In [22]:
def predict_tags(model, token_idxs_batch):
    """Performs predictions and transforms indices to tokens and tags."""
    
    tag_idxs_batch = model.predict(token_idxs_batch)
    tag_idxs_batch = np.argmax(tag_idxs_batch, axis=-1)
    tags_batch, tokens_batch = [], []
    for tag_idxs, token_idxs in zip(tag_idxs_batch, token_idxs_batch):
        tags, tokens = [], []
        for tag_idx, token_idx in zip(tag_idxs, token_idxs):
            tags.append(idx2tag[tag_idx])
            tokens.append(idx2token[token_idx])
        tags_batch.append(tags)
        tokens_batch.append(tokens)
    return tags_batch, tokens_batch
    
    
def eval_conll(model, tokens, tags, short_report=True):
    """Computes NER quality measures using CONLL shared task script."""
    
    y_true, y_pred = [], []
    for x_batch, y_batch in batches_generator(1, tokens, tags):
        tags_batch, tokens_batch = predict_tags(model, x_batch)
        if len(x_batch[0]) != len(tags_batch[0]):
            raise Exception("Incorrect length of prediction for the input, "
                            "expected length: %i, got: %i" % (len(x_batch[0]), len(tags_batch[0])))
        predicted_tags = []
        ground_truth_tags = []
        for gt_tag_idx, pred_tag, token in zip(y_batch[0], tags_batch[0], tokens_batch[0]): 
            if token != '<PAD>':
                ground_truth_tags.append(idx2tag[gt_tag_idx])
                predicted_tags.append(pred_tag)

        # We extend every prediction and ground truth sequence with 'O' tag
        # to indicate a possible end of entity.
        y_true.extend(ground_truth_tags + ['O'])
        y_pred.extend(predicted_tags + ['O'])
        
    results = precision_recall_f1(y_true, y_pred, print_results=True, short_report=short_report)
    return results

## Run your experiment

Create *BiLSTMModel* model with the following parameters:
 - *vocabulary_size* — number of tokens;
 - *n_tags* — number of tags;
 - *embedding_dim* — dimension of embeddings, recommended value: 200;
 - *n_hidden_rnn* — size of hidden layers for RNN, recommended value: 200;
 - *PAD_index* — an index of the padding token (`<PAD>`).

Set hyperparameters. You might want to start with the following recommended values:
- *batch_size*: 32;
- 4 epochs;
- starting value of *learning_rate*: 0.005
- *learning_rate_decay*: a square root of 2;
- *dropout_keep_probability*: try several values: 0.1, 0.5, 0.9.

However, feel free to conduct more experiments to tune hyperparameters and earn extra points for the assignment.

In [29]:
print("Vocabulary size is {}".format(len(token2idx)))
print("Number of tags is {}".format(len(tag2idx)))
print('Index of "<PAD>" token is {}'.format(token2idx['<PAD>']))

Vocabulary size is 20505
Number of tags is 21
Index of "<PAD>" token is 0


In [30]:
EMBEDDING_DIM = 300
RNN_HIDDEN = 100
DROPOUT_PROB = 0.3
N_EPOCHS = 64
tf.keras.backend.clear_session()
model = BiLSTMModel(len(token2idx), EMBEDDING_DIM, RNN_HIDDEN, len(tag2idx), DROPOUT_PROB)
dataset = tf.data.Dataset.from_generator(lambda: batches_generator(128, train_tokens, train_tags), (tf.int64, tf.int64), (tf.TensorShape([None, None]), tf.TensorShape([None, None]))).prefetch(tf.data.experimental.AUTOTUNE)
model.compile(loss=SparseCategoricalCrossentropy(from_logits=True), optimizer=Adam())
model.fit(dataset, epochs=N_EPOCHS)

Epoch 1/64
46/46 [==============================] - 7s 157ms/step - loss: 0.6259
Epoch 2/64
46/46 [==============================] - 5s 113ms/step - loss: 0.2376
Epoch 3/64
46/46 [==============================] - 5s 113ms/step - loss: 0.1871
Epoch 4/64
46/46 [==============================] - 5s 114ms/step - loss: 0.1533
Epoch 5/64
46/46 [==============================] - 5s 112ms/step - loss: 0.1315
Epoch 6/64
46/46 [==============================] - 5s 113ms/step - loss: 0.1229
Epoch 7/64
46/46 [==============================] - 5s 112ms/step - loss: 0.1134
Epoch 8/64
46/46 [==============================] - 5s 113ms/step - loss: 0.1044
Epoch 9/64
46/46 [==============================] - 5s 113ms/step - loss: 0.0970
Epoch 10/64
46/46 [==============================] - 5s 112ms/step - loss: 0.0869
Epoch 11/64
46/46 [==============================] - 5s 113ms/step - loss: 0.0750
Epoch 12/64
46/46 [==============================] - 5s 113ms/step - loss: 0.0621
Epoch 13/64
46/46 [======

Now let us see full quality reports for the final model on train, validation, and test sets. To give you a hint whether you have implemented everything correctly, you might expect F-score about 40% on the validation set.

**The output of the cell below (as well as the output of all the other cells) should be present in the notebook for peer2peer review!**

In [23]:
print('-' * 20 + ' Train set quality: ' + '-' * 20)
train_results = eval_conll(model, train_tokens, train_tags, short_report=False)

print('-' * 20 + ' Validation set quality: ' + '-' * 20)
validation_results = eval_conll(model, validation_tokens, validation_tags, short_report=False)

print('-' * 20 + ' Test set quality: ' + '-' * 20)
test_results = eval_conll(model, test_tokens, test_tags, short_report=False)

-------------------- Train set quality: --------------------
processed 105778 tokens with 4489 phrases; found: 4496 phrases; correct: 4466.

precision:  99.33%; recall:  99.49%; F1:  99.41

	     company: precision:   99.07%; recall:   99.38%; F1:   99.22; predicted:   645

	    facility: precision:   98.41%; recall:   98.73%; F1:   98.57; predicted:   315

	     geo-loc: precision:  100.00%; recall:   99.90%; F1:   99.95; predicted:   995

	       movie: precision:  100.00%; recall:  100.00%; F1:  100.00; predicted:    68

	 musicartist: precision:   98.72%; recall:   99.57%; F1:   99.14; predicted:   234

	       other: precision:   98.69%; recall:   99.21%; F1:   98.95; predicted:   761

	      person: precision:   99.89%; recall:   99.77%; F1:   99.83; predicted:   885

	     product: precision:   99.37%; recall:   99.69%; F1:   99.53; predicted:   319

	  sportsteam: precision:   99.08%; recall:   99.54%; F1:   99.31; predicted:   218

	      tvshow: precision:   98.21%; recall:  

### Conclusions

Could we say that our model is state of the art and the results are acceptable for the task? Definately, we can say so. Nowadays, Bi-LSTM is one of the state of the art approaches for solving NER problem and it outperforms other classical methods. Despite the fact that we used small training corpora (in comparison with usual sizes of corpora in Deep Learning), our results are quite good. In addition, in this task there are many possible named entities and for some of them we have only several dozens of trainig examples, which is definately small. However, the implemented model outperforms classical CRFs for this task. Even better results could be obtained by some combinations of several types of methods, e.g. see [this](https://arxiv.org/abs/1603.01354) paper if you are interested.